<a href="https://colab.research.google.com/github/Gaurav1401/Zenatix_Solutions/blob/main/FbProphet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this notebook, I am considering that the power consumption of all the AC's is independent of each other**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
from sklearn.metrics import r2_score

from multiprocessing import Pool, cpu_count
from fbprophet import Prophet

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
final = pd.read_csv("another_data.csv")

In [3]:
timestamp = final["Timestamp"]
final.drop("Timestamp", axis = "columns", inplace = True)

In [4]:
final[final < 0] = 0
final[final > 11.2] = 11.2

In [5]:
final.insert(0, "Timestamp", timestamp)

In [6]:
final.drop("index", axis = "columns", inplace = True)

In [7]:
final = final.set_index("Timestamp")

In [8]:
final.iloc[37440:-120, :].shape

(50280, 18)

**I am including the data of only last 5 days of the month of August as for the predictions of November month, the valeus won't be much dependent on the values of August Month**

In [27]:
train = final.iloc[37440:-120, :] 
test = final.iloc[-120:, :]

# **FbProphet implementation**

In [10]:
def run_prophet(timeseries):
    model = Prophet(uncertainty_samples=False) # won't include confidence intervals and will also save time
    model.fit(timeseries)
    future = model.make_future_dataframe(periods=120, include_history=False) #won't include training data in the output
    forecast = model.predict(future)
    return forecast

In [24]:
predictions = pd.DataFrame()

In [28]:
for i in test.columns:
  train1 = pd.DataFrame()
  train1 = train[[i]].reset_index()
  train1.columns = ['ds', 'y']


  with Pool(cpu_count()) as p:
      forecast1 = p.map(run_prophet, [train1])


  predictions[i] = forecast1[0]["yhat"]

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with ye

In [29]:
predictions.head()

,AC 1,AC 2,AC 3,AC 4,AC 5,AC 6,AC 7,AC 8,AC 9,AC 10,AC 11,AC 12,AC 13,AC 14,AC 15,AC 16,AC 17,AC 18
0,1.266718,0.912841,1.046615,0.993344,1.119411,0.465650,1.754878,1.372503,2.679313,1.382641,0.695030,1.620186,2.194366,2.465160,2.466734,2.296729,3.511975,4.751027
1,-0.280673,-0.416499,1.396213,0.316059,1.092195,0.318910,1.615329,1.657501,2.359466,1.367646,-0.420846,1.635156,2.197498,1.930894,1.386476,1.503817,3.329750,5.152081
2,-1.644433,-1.000411,0.436834,-0.177038,0.744560,-0.463826,0.485763,1.042704,2.108950,1.315866,-0.824734,1.399542,2.313744,1.735179,0.910105,1.892161,3.141210,5.521105
3,0.191466,-0.002428,0.816011,0.861494,1.138303,0.824653,1.679077,1.859607,2.597719,1.525780,0.101136,0.771683,1.837454,2.518524,1.036319,1.382586,3.174634,4.191033
4,-0.225175,0.802732,-0.013359,0.570476,1.038768,0.068914,0.848890,1.456298,2.113314,1.273457,-0.100290,0.977059,1.822462,2.330574,0.863157,1.391231,3.189672,4.428967


In [30]:
np.max(np.array(predictions)), np.min(np.array(predictions))

(5.521105433973937, -58.26643318519122)

**Max value completely makes sense as in the month of November we can't expect a lot of power consumption<br>
But Min value is negative which is not possible so such values should be replaced with 0**

In [31]:
predictions[predictions < 0] = 0

# **Evaluation**

In [33]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [34]:
# Evaluating using Mean Absolute Percentage Error
lst = []
for i in test.columns:
    error = mape(test[i], predictions[i])
    print(f"{i} ----> {error}")
    lst.append(error)

AC 1 ----> 99.50157938575369
AC 2 ----> 99.76738843350614
AC 3 ----> 97.88017789307112
AC 4 ----> nan
AC 5 ----> 89.98235196960525
AC 6 ----> nan
AC 7 ----> 96.42260496912614
AC 8 ----> nan
AC 9 ----> 86.23047348850169
AC 10 ----> 84.22953218715797
AC 11 ----> 99.56721278531462
AC 12 ----> 96.05054051562291
AC 13 ----> 97.37022934435001
AC 14 ----> 89.91378210346124
AC 15 ----> 97.85354004907343
AC 16 ----> 96.02730021372173
AC 17 ----> 91.23131176660627
AC 18 ----> 92.87016657169576


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

invalid value encountered in true_divide



In [35]:
test["AC 4"] += 1
test["AC 6"] += 1
test["AC 8"] += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [36]:
predictions["AC 4"] += 1
predictions["AC 6"] += 1
predictions["AC 8"] += 1

In [37]:
for i in ["AC 4", "AC 6", "AC 8"]:
    error = mape(test[i], predictions[i])
    print(f"{i} ----> {error}")

AC 4 ----> 52.960012153338056
AC 6 ----> 34.25682301075278
AC 8 ----> 60.55065544737527
